# Block 7 Übung 3: Selektion von Patientenkohorten nach Diagnose

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "lubridate", "ggplot2")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/"

## MIMIC III-Basisdaten laden

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_csv(paste(base_url, "mimic-iii-demo/PATIENTS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))

# Behandlungsfälle laden
mimic.admissions.raw <- read_csv(paste(base_url, "mimic-iii-demo/ADMISSIONS.csv", sep=""),
                                 col_types = cols(  row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))

# Intensivaufenthalte laden
mimic.icustays.raw <- read_csv(paste(base_url, "mimic-iii-demo/ICUSTAYS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), dbsource = col_character(), first_careunit = col_character(), last_careunit = col_character(), first_wardid = col_double(), last_wardid = col_double(), intime = col_datetime(format = ""), outtime = col_datetime(format = ""), los = col_double()))


## MIMIC III-Diagnosen laden

In [ ]:
# Diagnosen laden
mimic.diagnoses.raw <- read_csv(paste(base_url, "mimic-iii-demo/DIAGNOSES_ICD.csv", sep=""),
                                col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), seq_num = col_integer(), icd9_code = col_character()))

### Bezeichner ergänzen

In [ ]:
# Diagnosebezeichner laden
mimic.d_icd_diagnoses.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_ICD_DIAGNOSES.csv", sep=""),
                                      col_types = cols(row_id = col_integer(), icd9_code = col_character(), short_title = col_character(), long_title = col_character()))

# Diagnosebezeichner hinzufügen
mimic.diagnoses.annotated <- mimic.diagnoses.raw %>% inner_join(mimic.d_icd_diagnoses.raw %>% select(icd9_code, short_title, long_title), by = "icd9_code")

head(mimic.diagnoses.annotated)

## Beispiel: Diagnosekatalog nach Begriff "Sepsis" durchsuchen

In [ ]:
mimic.d_icd_diagnoses.raw %>% 
    filter(grepl("Sepsis", long_title))

## Aufenthalte mit Sepsis herausfiltern

In [ ]:
# Kohorte abfragen
cohort.sep <- mimic.diagnoses.annotated %>% 
    filter(grepl("Sepsis", long_title)) %>% # Diagnosen mit (Teil-)Begriff "Sepsis" im Langbezeichner filtern
    select(hadm_id) %>%                     # für die gefundenen Einträge nur die Fallnummer selektieren
    distinct() %>%                          # doppelte Fallnummern zusammenführen
    mutate(cohort_sep_flag = TRUE)          # Spalte mit Sepsis-Flag ergänzen

# Anzahl der Treffer ausgeben
cohort.sep %>% summarize(n = n())

# Erste 6 Treffer ausgeben
head(cohort.sep)

## Aufgabe: Filtern Sie je eine Kohorte mit akutem Herzinfarkt sowie akutem Nierenversagen heraus!

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) anwenden.
* zugehöriges Notebook: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb)

Legen Sie die Kohorte jeweils in einem eigenen Tibble nach dem folgenden Schema ab:
* Name des Tibbles: cohort.Kürzel (z.B. `cohort.ami` für acute myocardial infarction, `cohort.acf` für acute kidney failure)
* Flag-Spalte für die Erkrankung mit dem gleichen Kürzel (z.B. `cohort_ami_flag`, `cohort_acf_flag`)

Sonstige Tipps:
* in der `grepl()`-Funktion können Sie neben den reinen Suchbegriffen auch *regular Expressions* verwenden, um z.B. mehrere Schreibweisen eines Wort wahlweise zuzulassen. Einen Überblick zu den in RegExes verwendbaren Matches finden Sie auf Seite des des [RStudio-Cheat Sheets zum stringr-Package](https://evoldyn.gitlab.io/evomics-2018/ref-sheets/R_strings.pdf)
* auf [regex101.com](https://regex101.com/) können regular Expressions interaktiv getestet & debugged werden. Bitte denken Sie daran, dass es sich dabei um ein Internetangebot handelt & keine Texte mit personenbezogenen Daten für Tests dort verwendet werden dürfen.

In [ ]:
# Kohorte mit akutem Herzinfarkt (Acute myocardial infarction)


In [ ]:
# Kohorte mit akutem Nierenversagen (Acute kidney failure)


## Optionale Ergänzung: Visualisieren Sie die Demographie & Falldaten der Kohorten!

### Beispiel für die Sepsis-Kohorte

In [ ]:
# Kohorte um Fall- & Patientenstammdaten ergänzen
cases.sep <- cohort.sep %>%
    # ausgewählte Falldaten joinen
    inner_join(mimic.admissions.raw %>% select(subject_id, hadm_id, admittime, dischtime, discharge_location), by = "hadm_id") %>%
    # ausgewählte Patientenstammdaten joinen
    inner_join(mimic.patients.raw %>% select(subject_id, gender, dob), by = "subject_id") %>%
    # Aufenthaltsdauer & Alter berechnen
    mutate(length_of_stay_days = difftime(dischtime, admittime, units = "days"),
           age                 = as.integer(interval(dob, admittime) / years(1)))

head(cases.sep)

# Alter & Geschlecht ausgeben
options(repr.plot.width = 10, repr.plot.height = 2.5)
ggplot(data = cases.sep, aes(x = age, fill = gender)) +
    geom_histogram()

# Aufenthaltsdauer & Entlassart ausgeben
ggplot(data = cases.sep, aes(x = length_of_stay_days, y = discharge_location, fill = discharge_location)) +
    geom_point()

### Visualisierung für AMI-Kohorte

### Visualisierung für ACF-Kohorte